<a href="https://colab.research.google.com/github/hassanSattariNia/FederatedLearning/blob/main/FL-Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets torch evaluate

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,Trainer ,TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This be

In [3]:
from datasets import load_dataset
import evaluate
dataset = load_dataset("glue", "mrpc")

In [4]:
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

Train set size: 3668
Validation set size: 408
Test set size: 1725


In [5]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [16]:
import evaluate

# Function to compute multiple metrics
def compute_metrics(eval_pred):
    # Load metrics
    accuracy_metric = evaluate.load("accuracy")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    # Return the metrics
    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }


In [7]:

class Client:
  def __init__(self , id , model , train_dataset , eval_dataset):
    self.model = model
    self.train_dataset = train_dataset
    self.eval_dataset = eval_dataset
    self.trainer = None

  def print(self):
    print(f"client {self.id} , size of train:{len(self.train_dataset)} , size of validation:${len(self.eval_dataset)}")

  def train(self):
    training_args = TrainingArguments(
    output_dir="./results",  # Directory to save checkpoints and logs
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
    )


    self.trainer = Trainer(
        model=self.model,
        args=training_args,
        train_dataset = self.train_dataset,
        eval_dataset = self.eval_dataset ,
        compute_metrics = compute_metrics
    )

    self.trainer.train()

  def get_model_parameters(self):
    return self.model.state_dict()

  def update_model_parameters(self, new_params):
    self.model.load_state_dict(new_params)


In [8]:
class Server:
  def __init__(self , model):
    self.model = model

  def aggregate_parameters(self, client_parameters):
      aggregated_params = {}
      for key in client_parameters[0].keys():
          aggregated_params[key] = sum(client_param[key] for client_param in client_parameters) / len(client_parameters)
      return aggregated_params

  def update_model_parameters(self, new_params):
    self.model.load_state_dict(new_params)

  def get_model_parameters(self):
    return self.model.state_dict()


In [12]:
# split dataset

train_dataset = tokenized_dataset['train']
total_train_size = len(train_dataset)
mid_point = total_train_size // 2

# تقسیم بخش train
client1_train = train_dataset.select(range(mid_point)).select(range(1000))
client2_train = train_dataset.select(range(mid_point , total_train_size)).select(range(1000))

In [13]:
def federated_learning(num_rounds=3):

    client1 = Client(1,model,client1_train,tokenized_dataset['validation'])
    client2 = Client(2,model,client2_train,tokenized_dataset['validation'])
    server = Server(model)


    for round in range(num_rounds):
        print(f"Round {round + 1}")

        client1.train()
        client2.train()

        client_params = [client1.get_model_parameters(), client2.get_model_parameters()]

        aggregated_params = server.aggregate_parameters(client_params)

        server.update_model_parameters(aggregated_params)
        client1.update_model_parameters(aggregated_params)
        client2.update_model_parameters(aggregated_params)




In [ ]:
federated_learning(10)

Round 1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.536457,0.705882,0.692502,0.705882,0.635157


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.578289,0.693627,0.685662,0.693627,0.593068


Round 2


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
